<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

## Impact of denoising as one increases


In [1]:
#! lamin init --storage ./lamin-intro --schema bionty

import scanpy as sc
from scprint import scPrint
from scdataloader import Preprocessor
from scdataloader.utils import load_genes
import numpy as np
import anndata as ad
from huggingface_hub import hf_hub_download

from scprint.tasks import Denoiser, withknn
from scprint.model.utils import downsample_profile
from scipy.stats import spearmanr
from anndata import AnnData
from scprint.tasks.denoise import plot_cell_depth_wise_corr_improvement

import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

import torch

torch.set_float32_matmul_precision("medium")

→ connected lamindb: jkobject/scprint2


/pasteur/appa/homes/jkalfon/simpler_flash/src/simpler_flash/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/pasteur/appa/homes/jkalfon/simpler_flash/src/simpler_flash/layer_norm.py:1107: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [2]:
# model_checkpoint_file = hf_hub_download(
#    repo_id="jkobject/scPRINT", filename=f"v2-medium.ckpt"
# )
# model_checkpoint_file = ../data/
model_checkpoint_file = "../../../w937u4o1.ckpt"
# w937u4o1

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
model = scPrint.load_from_checkpoint(
    model_checkpoint_file, precpt_gene_emb=None, max_cont_len=10_000
)
model = model.to("cuda")

Gene position encoding has changed in the dataloader compared to last time, trying to revert


/pasteur/appa/homes/jkalfon/scdataloader/scdataloader/utils.py:427: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)


FYI: scPrint is not attached to a `Trainer`.


RuntimeError: Error(s) in loading state_dict for scPrint:
	Missing key(s) in state_dict: "expr_encoder.gene_encoder.embeddings.weight", "expr_encoder.expr_encoder.encoder.0.weight", "expr_encoder.expr_encoder.encoder.0.bias", "expr_encoder.expr_encoder.encoder.1.weight", "expr_encoder.expr_encoder.encoder.1.bias", "expr_encoder.expr_encoder.encoder.4.weight", "expr_encoder.expr_encoder.encoder.4.bias", "expr_encoder.expr_encoder.encoder.5.weight", "expr_encoder.expr_encoder.encoder.5.bias", "expr_encoder.expr_encoder.encoder.8.weight", "expr_encoder.expr_encoder.encoder.8.bias", "gene_encoder.embeddings.weight". 
	Unexpected key(s) in state_dict: "gene_encoder.0.embeddings.weight", "gene_encoder.1.weight", "gene_encoder.1.bias", "gene_encoder.3.weight", "gene_encoder.3.bias". 
	size mismatch for expr_encoder.encoder.0.weight: copying a param with shape torch.Size([512, 1]) from checkpoint, the shape in current model is torch.Size([768, 576]).
	size mismatch for expr_encoder.encoder.0.b

# run denoising as depth increase

In [ ]:
#import lamindb as ln

#res = ln.Artifact.filter(uid="vhwQPOwZ3kyGtOv00001")[0]
#print(res.key)
#print(res.path)
#print()
#res = ln.Artifact.filter(uid="Q1fKksCKtOAgigiT0001")[0]
#print(res.key)
#print(res.path)

# 2025-02-25/h5ad/Velocyto/Homo_sapiens/SRX24486462.h5ad
# /pasteur/appa/scratch/jkalfon/scprint2/.lamindb/vhwQPOwZ3kyGtOv00001.h5ad

# 2025-02-25/h5ad/Velocyto/Homo_sapiens/SRX22526970.h5ad
# /pasteur/appa/scratch/jkalfon/scprint2/.lamindb/Q1fKksCKtOAgigiT0001.h5ad

In [ ]:
LOC = "../../data/temp/"  #

datasets = {
    "retina": "https://datasets.cellxgene.cziscience.com/53bd4177-79c6-40c8-b84d-ff300dcf1b5b.h5ad",  # gNNpgpo6gATjuxTE7CCp
    "kidney": "https://datasets.cellxgene.cziscience.com/01bc7039-961f-4c24-b407-d535a2a7ba2c.h5ad",
    "glio_smart_highdepth": "https://datasets.cellxgene.cziscience.com/6ec440b4-542a-4022-ac01-56f812e25593.h5ad",  # s8x0Idi587LQtXCo0Pif
    "lung_smart": "https://datasets.cellxgene.cziscience.com/6ebba0e0-a159-406f-8095-451115673a2c.h5ad",
    # additional (see origin from block above)
    "SRX24486462": None,
    "SRX22526970": None,
    #
}

In [ ]:
adata = sc.read(LOC + "lung_smart" + ".h5ad")


In [ ]:
adata = sc.read(LOC + "lung_smart" + ".h5ad")
print((adata.X > 0).sum(1).mean(), adata.X.sum(1).mean(), adata.shape[0])
plt.hist(np.array(adata.X.sum(1)).T[0], bins=50)
plt.title("reads per cells")
plt.show()
plt.hist(np.array(adata.X.sum(0))[0], bins=50)
plt.yscale("log")
plt.title("reads per genes")
plt.show()
# 4x

In [ ]:
adata = sc.read(LOC + "glio_smart_highdepth" + ".h5ad")
print((adata.X > 0).sum(1).mean(), adata.X.sum(1).mean(), adata.shape[0])
plt.hist(np.array(adata.X.sum(1)).T[0], bins=50)
plt.title("reads per cells")
plt.show()
plt.hist(np.array(adata.X.sum(0))[0], bins=50)
plt.yscale("log")
plt.title("reads per genes")
plt.show()
# 14x

In [ ]:
adata = sc.read(LOC + "SRX24486462" + ".h5ad")
print((adata.X > 0).sum(1).mean(), adata.X.sum(1).mean(), adata.shape[0])
plt.hist(np.array(adata.X.sum(1)).T[0], bins=50)
plt.title("reads per cells")
plt.show()
plt.hist(np.array(adata.X.sum(0))[0], bins=50)
plt.yscale("log")
plt.title("reads per genes")
plt.show()
# 7x

In [ ]:
adata = sc.read(LOC + "SRX22526970" + ".h5ad")
print((adata.X > 0).sum(1).mean(), adata.X.sum(1).mean(), adata.shape[0])
plt.hist(np.array(adata.X.sum(1)).T[0], bins=50)
plt.title("reads per cells")
plt.show()
plt.hist(np.array(adata.X.sum(0))[0], bins=50)
plt.title("reads per genes")
plt.yscale("log")
plt.show()
# 5x

In [ ]:
adata = sc.read(LOC + "kidney" + ".h5ad")
print((adata.X > 0).sum(1).mean(), adata.X.sum(1).mean(), adata.shape[0])
plt.hist(np.array(adata.X.sum(1)).T[0], bins=50)
plt.title("reads per cells")
plt.show()
plt.hist(np.array(adata.X.sum(0))[0], bins=50)
plt.title("reads per genes")
plt.yscale("log")
plt.show()
# 1x

In [ ]:
res = {}
for name, url in datasets.items():
    print("doing ", name)

    adata = sc.read(LOC + name + ".h5ad", backup_url=url)
    max_len = 3000 if (adata.X > 0).sum(1).mean() < 3_000 else 5000
    preprocessor = Preprocessor(
        force_preprocess=True,
        skip_validate=True,
        # drop_non_primary=False,
        do_postp=False,
    )
    print("")
    adata = adata[adata.X.sum(1) > 1000]
    print(adata.X.sum(1).mean())
    adata = preprocessor(adata)
    adata.layers["true"] = adata.X.copy()
    denoise = Denoiser(
        batch_size=40 if (adata.X > 0).sum(1).mean() < 3_000 else 20,
        max_len=max_len,
        max_cells=5_000,
        doplot=False,
        num_workers=8,
        predict_depth_mult=10,
        downsample_expr=0.7,
        additional_info=True,
        apply_zero_pred=True,
    )
    res["scprint_" + name], idx, nadata = denoise(model, adata)
    print(res["scprint_" + name])

    # doing magic
    subadata = (
        adata[idx, adata.var.index.isin(model.genes) & adata.var.highly_variable].copy()
        if idx is not None
        else adata[
            :, adata.var.index.isin(model.genes) & adata.var.highly_variable
        ].copy()
    )
    print(subadata.shape)
    subadata.X = downsample_profile(torch.Tensor(subadata.X.toarray()), 0.7)
    denoised_adata = sc.external.pp.magic(
        subadata.copy(), copy=True, n_jobs=10, solver="approximate", verbose=True
    )

    reco = denoised_adata.X
    true = subadata.layers["true"].toarray()
    noisy = subadata.X
    corr_coef, p_value = spearmanr(
        np.vstack([reco[true != 0], noisy[true != 0], true[true != 0]]).T
    )
    res["magic_" + name] = {
        "reco2noisy": corr_coef[0, 1],
        "reco2full": corr_coef[0, 2],
        "noisy2full": corr_coef[1, 2],
    }
    if reco.shape[0] <= 3000:
        indices = np.random.choice(reco.shape[0], 3000, replace=False)
        reco = reco[indices]
        noisy = noisy[indices]
        true = true[indices]
    corr, p_value = spearmanr(
        np.vstack(
            [
                reco.flatten(),
                noisy.flatten(),
                true.flatten(),
            ]
        ).T
    )
    m = {
        "reco2full": corr[0, 2],
        "noisy2full": corr[1, 2],
    }
    print("corr with zeros: ")
    print(m)
    cell_wise = np.array(
        [
            spearmanr(reco[i][true[i] != 0], true[i][true[i] != 0])[0]
            for i in range(reco.shape[0])
        ]
    )
    torm = np.array(
        [
            spearmanr(noisy[i][true[i] != 0], true[i][true[i] != 0])[0]
            for i in range(reco.shape[0])
        ]
    )
    cell_wise -= torm
    cell_wise_zero = np.mean(
        [spearmanr(reco[i], true[i])[0] for i in range(reco.shape[0])]
    )
    print("cell_wise self corr (reco, noisy, true)")
    print(
        {
            "cell_wise_w_zero": cell_wise_zero,
            "cell_wise_to_noisy": np.mean(cell_wise),
        }
    )
    print("depth-wise plot")
    plot_cell_depth_wise_corr_improvement(cell_wise, (true > 0).sum(1))
    print(res["magic_" + name])
    print("--------------------------------")
    print("")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

# Create a DataFrame
# from prevmodel
# data = {
#    'depth': [2736, 1888, 388362, 10360],
#    'magic': [30.4, 26.3, 6.4, 27.0],
#    'scprint': [19.3, 21.5, 33.2, 36.1]
# }
data = {
    "depth": [2736, 1888, 388362, 10360, 21693, 38469],
    "magic": [28.5, 26.3, 6.4, 22.0, 49, 39],
    "scprint": [29.5, 28.5, 35.7, 36.3, 43.6, 31],
}
df = pd.DataFrame(data)

# Convert depth to log scale
df["log_depth"] = np.log(df["depth"])


# Define a linear function for curve fitting
def linear_func(x, a, b):
    return a * x + b


# Fit the linear curves on log scale
popt_magic, _ = curve_fit(linear_func, df["log_depth"], df["magic"])
popt_scprint, _ = curve_fit(linear_func, df["log_depth"], df["scprint"])

# Plot the data points
plt.scatter(df["log_depth"], df["magic"], label="Magic", color="blue")
plt.scatter(df["log_depth"], df["scprint"], label="scPRINT", color="red")

# Plot the fitted linear curves
x_values = np.linspace(min(df["log_depth"]), max(df["log_depth"]), 100)
plt.plot(
    x_values,
    linear_func(x_values, *popt_magic),
    label="Linear Fit (Magic)",
    color="blue",
    linestyle="--",
)
plt.plot(
    x_values,
    linear_func(x_values, *popt_scprint),
    label="Linear Fit (scPRINT)",
    color="red",
    linestyle="--",
)

# Add labels and legend
plt.xlabel("Log Depth")
plt.ylabel("Values")
plt.legend()
plt.title("Log Depth vs Magic and scPRINT with Linear Fits")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

# Create a DataFrame
# from prevmodel
# data = {
#    'depth': [2736, 1888, 388362, 10360],
#    'magic': [30.4, 26.3, 6.4, 27.0],
#    'scprint': [19.3, 21.5, 33.2, 36.1]
# }
data = {
    "depth": [2736, 1888, 388362, 10360],
    "magic": [30.5, 26.3, 6.4, 22.0],
    "scprint": [29.5, 26.5, 35.7, 36.3],
}
df = pd.DataFrame(data)

# Convert depth to log scale
df["log_depth"] = np.log(df["depth"])


# Define a linear function for curve fitting
def linear_func(x, a, b):
    return a * x + b


# Fit the linear curves on log scale
popt_magic, _ = curve_fit(linear_func, df["log_depth"], df["magic"])
popt_scprint, _ = curve_fit(linear_func, df["log_depth"], df["scprint"])

# Plot the data points
plt.scatter(df["log_depth"], df["magic"], label="Magic", color="blue")
plt.scatter(df["log_depth"], df["scprint"], label="scPRINT", color="red")

# Plot the fitted linear curves
x_values = np.linspace(min(df["log_depth"]), max(df["log_depth"]), 100)
plt.plot(
    x_values,
    linear_func(x_values, *popt_magic),
    label="Linear Fit (Magic)",
    color="blue",
    linestyle="--",
)
plt.plot(
    x_values,
    linear_func(x_values, *popt_scprint),
    label="Linear Fit (scPRINT)",
    color="red",
    linestyle="--",
)

# Add labels and legend
plt.xlabel("Log Depth")
plt.ylabel("Values")
plt.legend()
plt.title("Log Depth vs Magic and scPRINT with Linear Fits")
plt.show()


In [ ]:
adata = sc.read(LOC + "lung_smart" + ".h5ad")

In [ ]:
preprocessor = Preprocessor(
    force_preprocess=True, skip_validate=True, drop_non_primary=False, do_postp=False
)
preprocessor = Preprocessor(do_postp=False)
adata = preprocessor(adata)
adata.layers["true"] = adata.X.copy()

In [ ]:
denoise = Denoiser(
    batch_size=40 if adata.X.sum(1).mean() < 50_000 else 20,
    max_len=8_000,
    max_cells=100_000,
    doplot=False,
    num_workers=8,
    predict_depth_mult=5,
    downsample_expr=0.7,
)

In [ ]:
_, idx, nadata = denoise(model, adata)

In [ ]:
nadata = nadata[:, nadata.var.index.isin(adata.var.index)]
adata = adata[:, adata.var.index.isin(nadata.var.index)]

In [ ]:
# Get min and max from adata
adata_min = np.array([1] * adata.X.shape[0])
adata_max = adata.X.max(1).data

# Get min and max for each row in nadata
nadata_min = nadata.layers["scprint_mu"].data.reshape(adata.X.shape[0], -1).min(axis=1)
nadata_max = nadata.layers["scprint_mu"].data.reshape(adata.X.shape[0], -1).max(axis=1)

# Rescale each row to match adata range
# Reshape the min/max arrays to match the matrix dimensions for broadcasting
nadata_min = nadata_min.reshape(-1, 1)
nadata_max = nadata_max.reshape(-1, 1)
adata_min = adata_min.reshape(-1, 1)
adata_max = adata_max.reshape(-1, 1)

# Perform the rescaling with properly shaped arrays
nadata.layers["scprint_mu"].data = (
    (nadata.layers["scprint_mu"].data.reshape(adata.X.shape[0], -1) - nadata_min)
    / (nadata_max - nadata_min)
    * (adata_max - adata_min)
    + adata_min
).flatten()


In [ ]:
# Create a mask for where scprint_mu is 0
zero_mask = nadata.layers["scprint_mu"] == 0
# Replace zeros with corresponding values from adata.X
nadata.layers["scprint_mu"][zero_mask] = adata.X[zero_mask]


In [ ]:
nadata.obs["cell_type"] = adata.obs["cell_type"].tolist()

In [ ]:
# Create UMAP visualization of scprint_mu layer
import scanpy as sc

# Compute UMAP
# Compute PCA first
sc.pp.pca(nadata, layer="scprint_mu", n_comps=40)

sc.pp.neighbors(nadata, use_rep="X_pca", n_neighbors=15)
sc.tl.umap(nadata, n_components=2, min_dist=0.2, spread=1.5)
nadata.obs["cell_type"] = adata.obs["cell_type"].tolist()

# Plot UMAP colored by cell type
sc.pl.umap(
    nadata,
    color="cell_type",
    title="UMAP of scPRINT denoised data",
    frameon=False,
    legend_fontsize=8,
)


In [ ]:
sc.pl.umap(
    nadata,
    color="cell_type",
    title="UMAP of scPRINT denoised data",
    frameon=False,
    legend_fontsize=8,
)

In [ ]:
sc.pl.umap(
    nadata,
    color="cell_type",
    title="UMAP of scPRINT denoised data",
    frameon=False,
    legend_fontsize=8,
)

In [ ]:
adata.X = downsample_profile(torch.Tensor(adata.X.toarray()), 0.7).numpy()

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata, use_rep="X_pca")
sc.tl.umap(adata)

# Plot UMAP colored by cell type
sc.pl.umap(
    adata,
    color="cell_type",
    title="UMAP of noisy data",
    frameon=False,
    legend_fontsize=8,
    palette=nadata.uns["cell_type_colors"],
)

In [ ]:
adata

In [ ]:
adata.layers["true"].sum(1).mean(), adata.X.sum(1).mean()

In [ ]:
adata.obsm["X_pca"] = sc.pp.pca(adata.layers["true"])
sc.pp.neighbors(adata, use_rep="X_pca")
sc.tl.umap(adata)

# Plot UMAP colored by cell type
sc.pl.umap(
    adata,
    color="cell_type",
    title="UMAP of ground truth data",
    frameon=False,
    legend_fontsize=8,
    palette=nadata.uns["cell_type_colors"],
)

In [ ]:
scprint vs magic (full vs rare)
33.6 vs 37.7 - 28.2 vs 26.4
30 vs 20 - 28.5 vs 17.5
22.3 vs 23.5 - 23.4 vs 5.0

## with MAGIC rare cells

In [ ]:
ct = "pericyte"
subadata.obs.cell_type.value_counts()
# microglial cell, microfold cell of epithelium of small intestine, mesangial cell, pericyte

In [ ]:
reco = denoised_adata[subadata.obs.cell_type == ct].X
true = subadata[subadata.obs.cell_type == ct].layers["true"].toarray()
noisy = subadata[subadata.obs.cell_type == ct].X
corr_coef, p_value = spearmanr(
    np.vstack([reco[true != 0], noisy[true != 0], true[true != 0]]).T
)
{
    "reco2noisy": corr_coef[0, 1],
    "reco2full": corr_coef[0, 2],
    "noisy2full": corr_coef[1, 2],
}

## scPRINT rare cells

In [ ]:
denoise = Denoiser(
    batch_size=40,
    max_len=5000,
    max_cells=200,
    doplot=False,
    predict_depth_mult=6,
    downsample=0.7,
    how="most var",
)
metrics, idx, genes, expr = denoise(model, adata[adata.obs.cell_type == ct])
metrics